In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
crashes = pd.read_csv('Crashes.csv')
crashes_columns = []
for c in crashes.columns:

  crashes_columns. append(c)
  
id_columns = []
for c in crashes_columns:

  if c[-2:] == "ID":
    id_columns.append(c)

cd_columns = []
for c in crashes_columns:

  if c[-2:] == "CD":
    cd_columns.append(c)
    
crashes_new = crashes.drop(id_columns, axis=1)
crashes_new = crashes_new.drop(cd_columns, axis=1)

crashes_portland = crashes_new.dropna(subset = 'CITY_SECT_NM')
crashes_portland = crashes_portland[crashes_portland['CITY_SECT_NM'].str.contains('Portland')]

unused_columns = ['CRASH_DT', 'X', 'Y', 'SER_NO', 'HWY_NO', 'HWY_SFX_NO', 'LRS_VAL', 'AGY_ST_NO', 'ISECT_AGY_ST_NO', 'ISECT_SEQ_NO', 'FROM_ISECT_DSTNC_QTY', 'GIS_PRC_DT', # Irrelevant Information
                      'CNTY_NM', 'CITY_SECT_NM', 'URB_AREA_LONG_NM', 'CRASH_HR_LONG_DESC', 'POP_RNG_MED_DESC', 'EFFECTV_DT', 'GIS_JURIS', # Redundant Information
                      'LAT_DEG_NO', 'LAT_MINUTE_NO', 'LAT_SEC_NO', 'LONGTD_DEG_NO', 'LONGTD_MINUTE_NO', 'LONGTD_SEC_NO', #Latitude/Longitude
                      'SEG_PT_LRS_MEAS', 'SPECL_JRSDCT_LONG_DESC', 'RECRE_RD_NM', 'ISECT_RECRE_RD_NM', # No values - All missing data
                      'UNLOCT_FLG', 'TRAF_CNTL_FUNC_FLG', 'UGB', 'MPA', # Flags that have only one value
                      'CRASH_CAUSE_2_LONG_DESC', 'CRASH_CAUSE_3_LONG_DESC', 'CRASH_EVNT_2_LONG_DESC', 'CRASH_EVNT_3_LONG_DESC' # Extraneous crash event descriptions
                      ]

crashes_portland.drop(unused_columns, axis = 1)



C:\Users\miles\AppData\Local\Temp\ipykernel_14328\3592173246.py:4: DtypeWarning: Columns (17,22,26,43,44,45,46,48) have mixed types. Specify dtype option on import or set low_memory=False.
  crashes = pd.read_csv('Crashes.csv')


,CRASH_HR_NO,FC_DESC,NHS_FLG,RTE_NM,HWY_MED_NM,RDWY_NO,HWY_COMPNT_LONG_DESC,MLGE_TYP_LONG_DESC,RD_CON_NO,MP_NO,...,TOT_MOTOR_FATAL,TOT_MOTOR_INJ_A,TOT_PED_INJ_A_CNT,TOT_PEDCYCL_INJ_A_CNT,TOT_MTRCYCL_FATAL_CNT,TOT_MTRCYCL_INJ_A_CNT,HIGHEST_INJ_SVRTY_DESC,MJ_INVLV_FLG,TOT_MTRCYCL_CNT,TOT_MOTOR_CNT
0,15,URBAN PRINCIPAL ARTERIAL - INTERSTATE,1,IS 205,EAST PORTLAND FREEWAY,2.0,Mainline State Highway,Regular Mileage,NaN,19.08,...,0,0,0,0,0,0,No Apparent Injury/PDO Crash (O),NaN,0,2
1,7,URBAN PRINCIPAL ARTERIAL - INTERSTATE,1,IS 205,EAST PORTLAND FREEWAY,1.0,Connection,Regular Mileage,2.0,19.12,...,0,0,0,0,0,0,No Apparent Injury/PDO Crash (O),NaN,0,2
2,17,URBAN PRINCIPAL ARTERIAL - OTHER,1,US 26,MT. HOOD,1.0,Mainline State Highway,Regular Mileage,NaN,6.18,...,0,0,0,0,0,0,No Apparent Injury/PDO Crash (O),NaN,0,2
3,15,URBAN PRINCIPAL ARTERIAL - INTERSTATE,1,IS 205,EAST PORTLAND FREEWAY,2.0,Mainline State Highway,Regular Mileage,NaN,19.36,...,0,0,0,0,0,0,No Apparent Injury/PDO Crash (O),NaN,0,2
4,8,URBAN PRINCIPAL ARTERIAL - OTHER,1,US 26,MT. HOOD,1.0,Mainline State Highway,Regular Mileage,NaN,5.59,...,0,0,0,0,0,0,No Apparent Injury/PDO Crash (O),NaN,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
338977,13,URBAN MINOR ARTERIAL,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,No Apparent Injury/PDO Crash (O),0.0,0,2
338978,9,URBAN PRINCIPAL ARTERIAL - OTHER,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,No Apparent Injury/PDO Crash (O),0.0,0,2
338981,16,URBAN PRINCIPAL ARTERIAL - INTERSTATE,1,I-205,EAST PORTLAND FREEWAY,1.0,Mainline State Highway,Regular Mileage,NaN,25.25,...,0,0,0,0,0,0,Possible Injury Crash (C),0.0,0,2
338983,17,URBAN PRINCIPAL ARTERIAL - OTHER,1,US 26,MT. HOOD,1.0,Mainline State Highway,Regular Mileage,NaN,2.73,...,0,0,0,0,0,0,No Apparent Injury/PDO Crash (O),0.0,0,2


In [7]:
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
from sklearn.preprocessing import MinMaxScaler
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go


In [14]:

yrs = crashes_portland[crashes_portland['CRASH_YR_NO'].isin(range(2017, 2022))]
speed = yrs[['LAT_DD', 'LONGTD_DD', 'CRASH_SPEED_INVLV_FLG']]
#speed = speed[speed['CRASH_SPEED_INVLV_FLG'] == 1]

# Normalizing the data using MinMaxScaler, which is better for geographic data
'''scaler = MinMaxScaler()
scaled_numerical_features = scaler.fit_transform(speed)
scaled_data = pd.DataFrame(scaled_numerical_features, columns = ['LAT_DD', 'LONGTD_DD', 'CRASH_SPEED_INVLV_FLAG'])'''

"scaler = MinMaxScaler()\nscaled_numerical_features = scaler.fit_transform(speed)\nscaled_data = pd.DataFrame(scaled_numerical_features, columns = ['LAT_DD', 'LONGTD_DD', 'CRASH_SPEED_INVLV_FLAG'])"

In [15]:
kmeans = KMeans(n_clusters = 150, random_state = 42)
y_pred = kmeans.fit_predict(speed[['LAT_DD', 'LONGTD_DD']])
speed['cluster'] = y_pred

C:\Users\miles\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

C:\Users\miles\AppData\Local\Temp\ipykernel_14328\4066137010.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [10]:
fig = px.scatter_mapbox(speed, 
                        lat="LAT_DD", 
                        lon="LONGTD_DD",
                        color='cluster', 
                        zoom=10)

fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [18]:
cluster_counts_speed = speed.groupby(['cluster', 'CRASH_SPEED_INVLV_FLG']).size().unstack().fillna(0)
cluster_proportions_speed = cluster_counts_speed.div(cluster_counts_speed.sum(axis=1), axis=0)


print(cluster_proportions_speed.sort_values(by = 0))

CRASH_SPEED_INVLV_FLG         0         1
cluster                                  
136                    0.676471  0.323529
48                     0.695652  0.304348
117                    0.790698  0.209302
55                     0.839286  0.160714
25                     0.855556  0.144444
...                         ...       ...
51                     0.978456  0.021544
46                     0.978836  0.021164
100                    0.979021  0.020979
65                     0.983471  0.016529
89                     0.991254  0.008746

[150 rows x 2 columns]


In [20]:
high_concentration_clusters_speed = cluster_proportions_speed[1.0] > 0.12  # Define a threshold
high_concentration_cluster_ids_speed = high_concentration_clusters_speed[high_concentration_clusters_speed].index

print(high_concentration_cluster_ids_speed)

Index([25, 48, 55, 56, 75, 117, 124, 136], dtype='int32', name='cluster')


In [22]:
speed_cluster = speed[speed['cluster'].isin(high_concentration_cluster_ids_speed)]

fig = px.scatter_mapbox(speed_cluster, 
                        lat="LAT_DD", 
                        lon="LONGTD_DD",
                        color='cluster', 
                        zoom=10)

fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()